<a href="https://colab.research.google.com/github/rpizarrog/machine_learning_r_python_casos_de_estudio/blob/main/notebook_Python/LDA_datos_clientes_aseguradora_MANUAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo LDA con datos de clientes de aseguradora

# Objetivo

Implementar paso a paso modelo LDA con datos inicales que simulan valores de clientes de una aseguradora.

# Inicializar datos


In [4]:
import pandas as pd
import numpy as np
# Evitar notación científica y fijar decimales
np.set_printoptions(suppress=True, precision=4)

# Datos iniciales
valores = {
    "id": [1,2,3,4,5,6,7,8,9,10],
    "edad": [25,28,32,29,35,50,55,60,48,52],
    "ingresos": [15000,18000,20000,16000,21000,28000,30000,35000,27000,29000],
    "hijos": [0,1,2,1,2,3,2,4,3,2],
    "gastos_medicos": [2000,2500,3000,2200,3100,6000,7000,8000,5500,6200],
    "ejercicio": [6,5,3,4,2,1,0,1,2,0],
    "imc": [22,24,27,23,28,30,31,29,28,32],
    "clase": ["Joven","Joven","Joven","Joven","Joven","Mayor","Mayor","Mayor","Mayor","Mayor"]
}
datos = pd.DataFrame(valores)
print(datos.head())



   id  edad  ingresos  hijos  gastos_medicos  ejercicio  imc  clase
0   1    25     15000      0            2000          6   22  Joven
1   2    28     18000      1            2500          5   24  Joven
2   3    32     20000      2            3000          3   27  Joven
3   4    29     16000      1            2200          4   23  Joven
4   5    35     21000      2            3100          2   28  Joven


# Separar variables numéricas y la clase


In [5]:
numericas = datos[["edad","ingresos","hijos","gastos_medicos","ejercicio","imc"]].values
clase = datos["clase"].values
clases = np.unique(clase)
print(numericas)
print(clases)


[[   25 15000     0  2000     6    22]
 [   28 18000     1  2500     5    24]
 [   32 20000     2  3000     3    27]
 [   29 16000     1  2200     4    23]
 [   35 21000     2  3100     2    28]
 [   50 28000     3  6000     1    30]
 [   55 30000     2  7000     0    31]
 [   60 35000     4  8000     1    29]
 [   48 27000     3  5500     2    28]
 [   52 29000     2  6200     0    32]]
['Joven' 'Mayor']


# Calcular medias por clase


In [24]:
# Diccionario para guardar medias por clase
medias_clase = {}

for c in clases:
    Xc = numericas[clase==c]
    medias_clase[c] = Xc.mean(axis=0)

print("Medias por clase:")
for c, mu in medias_clase.items():
    print(c, mu)



Medias por clase:
Joven [   29.8 18000.      1.2  2560.      4.     24.8]
Mayor [   53.  29800.      2.8  6540.      0.8    30. ]


# Construcción de SW


In [25]:
# Inicializar matriz SW
d = numericas.shape[1]   # número de variables numéricas, 6
SW = np.zeros((d,d))

for c in clases:
    Xc = numericas[clase==c]
    mu_c = medias_clase[c]
    # Matriz centrada de la clase
    Mc = Xc - mu_c  # Matriz de cada clase
    # Acumular productos exteriores
    SW += Mc.T @ Mc # Se construye el producto de ambas matrices se multiplica la matriz Mc por su transpuesta. El símbolo @ significa multiplicar matrices y Mc.T es la matriz transpuesta de M.
print("\nMatriz de dispersión dentro de clases SW:")
print(np.round(SW,2))







Matriz de dispersión dentro de clases SW:
[[     146.8    93000.        18.2    25160.       -30.        40.8]
 [   93000.  64800000.     13800.  16740000.    -16200.     25000. ]
 [      18.2    13800.         5.6     2880.        -3.2        4.2]
 [   25160.  16740000.      2880.   4764000.     -3860.      5360. ]
 [     -30.    -16200.        -3.2    -3860.        12.8      -20. ]
 [      40.8    25000.         4.2     5360.       -20.        36.8]]


# Construcción SB


In [26]:
# Calcular SB
cols = ["edad","ingresos","hijos","gastos_medicos","ejercicio","imc"]
mu_global = numericas.mean(axis=0) # Variables previamente identificadas
SB = np.zeros((d, d)) # Inicializar la matriz vacía con ceros

print ("Media global de variables")
print(mu_global)

# Se recorre cada clase ("Joven", "Mayor")
# Extraer todas las observaciones de la clase actual (submatriz nk × d)
# Calcular el vector de medias de esa clase (d valores)
# Contar cuántas observaciones hay en la clase (nk)
# Construir Vector columna (d × 1) con la diferencia entre media de clase y media global
# Acumular en SB: nk * producto exterior diff * diff^T (d × d)
# Presentar la matriz SB en data.frame
# Vector columna (d × 1) con la diferencia entre media de clase y media global

for c in clases:
    Xc = numericas[clase == c]
    mu_c = Xc.mean(axis=0)
    nk = Xc.shape[0]
    diff = (mu_c - mu_global).reshape(-1, 1)
    SB += nk * (diff @ diff.T)

SB_df = pd.DataFrame(np.round(SB, 2), index=cols, columns=cols)
print("\nMatriz de dispersión entre clases SB (redondeada):")
print(SB_df.to_string())



Media global de variables
[   41.4 23900.      2.   4550.      2.4    27.4]

Matriz de dispersión entre clases SB (redondeada):
                    edad     ingresos    hijos  gastos_medicos  ejercicio       imc
edad              1345.6     684400.0     92.8        230840.0     -185.6     301.6
ingresos        684400.0  348100000.0  47200.0     117410000.0   -94400.0  153400.0
hijos               92.8      47200.0      6.4         15920.0      -12.8      20.8
gastos_medicos  230840.0  117410000.0  15920.0      39601000.0   -31840.0   51740.0
ejercicio         -185.6     -94400.0    -12.8        -31840.0       25.6     -41.6
imc                301.6     153400.0     20.8         51740.0      -41.6      67.6


# Vector de diferencias de medias entre clases




In [27]:
mu_J = numericas[clase=="Joven"].mean(axis=0)
mu_M = numericas[clase=="Mayor"].mean(axis=0)
d = (mu_M - mu_J)  # vector 6x1 (lo dejamos como 1D y lo re-formamos cuando se necesite)

print("Media Joven (mu_J):", np.round(mu_J, 4))
print("Media Mayor (mu_M):", np.round(mu_M, 4))
print("d = mu_M - mu_J   :", np.round(d,    4), "\n")


Media Joven (mu_J): [   29.8 18000.      1.2  2560.      4.     24.8]
Media Mayor (mu_M): [   53.  29800.      2.8  6540.      0.8    30. ]
d = mu_M - mu_J   : [   23.2 11800.      1.6  3980.     -3.2     5.2] 



# Sistema de ecuaciones


In [28]:
w = np.linalg.solve(SW, d)     # w = SW^{-1} d

print("Solución w (sin normalizar):")
print(np.round(w, 6))

# Normalizar w (opcional, para reportar)
w_norm = w / np.linalg.norm(w)
print("\nw normalizada (||w||=1):")
print(np.round(w_norm, 4))


Solución w (sin normalizar):
[14.672  -0.0175  9.2892 -0.0092 35.4655 15.3361]

w normalizada (||w||=1):
[ 0.3463 -0.0004  0.2193 -0.0002  0.8372  0.362 ]


# Razón de Fisher


In [18]:
numerador = float(w.T @ SB @ w)   # w^T SB w
denominador = float(w.T @ SW @ w)   # w^T SW w
J = numerador / denominador               # razón de Fisher

print("Numerador (w^T SB w)      :", numerador)
print("Denominador (w^T SW w)    :", denominador)
print("Razón de Fisher J(w)      :", J)

Numerador (w^T SB w)      : 15243.327817868909
Denominador (w^T SW w)    : 78.08540918217618
Razón de Fisher J(w)      : 195.2135229554302


# D1


In [38]:
import numpy as np

X = numericas
print(X)

# Aseguramos que w_norm está normalizado
w_norm = w / np.linalg.norm(w)
print(w_norm)

# Proyección de cada fila de X sobre w_norm
D1 = X @ w_norm   # equivalente a X %*% w_norm en R
print("\nD1 (coordenadas en la primera dimensión LDA):")
print(np.round(D1, 4))


[[   25 15000     0  2000     6    22]
 [   28 18000     1  2500     5    24]
 [   32 20000     2  3000     3    27]
 [   29 16000     1  2200     4    23]
 [   35 21000     2  3100     2    28]
 [   50 28000     3  6000     1    30]
 [   55 30000     2  7000     0    31]
 [   60 35000     4  8000     1    29]
 [   48 27000     3  5500     2    28]
 [   52 29000     2  6200     0    32]]
[ 0.3463 -0.0004  0.2193 -0.0002  0.8372  0.362 ]

D1 (coordenadas en la primera dimensión LDA):
[15.0056 14.8009 14.8812 14.8407 15.0096 16.7851 16.7779 16.7754 16.7279
 16.6882]


# Calcular la matriz centrada

In [42]:
import numpy as np

# Ya tienes X como variables numéricas
# Ejemplo: X = datos[["edad","ingresos","hijos","gastos_medicos","ejercicio","imc"]].values
X = numericas
# 1) Calcular media global por columna (como colMeans en R)
mu_global = X.mean(axis=0)

# 2) Restar la media a cada columna (centrar los datos)
Xc = X - mu_global   # equivalente a scale(X, center=TRUE, scale=FALSE)

# 3) Mostrar resultados
print("Media global de cada variable:", np.round(mu_global, 4))
print("\nMatriz centrada (Xc):")
print(np.round(Xc, 2))


Media global de cada variable: [   41.4 23900.      2.   4550.      2.4    27.4]

Matriz centrada (Xc):
[[  -16.4 -8900.     -2.  -2550.      3.6    -5.4]
 [  -13.4 -5900.     -1.  -2050.      2.6    -3.4]
 [   -9.4 -3900.      0.  -1550.      0.6    -0.4]
 [  -12.4 -7900.     -1.  -2350.      1.6    -4.4]
 [   -6.4 -2900.      0.  -1450.     -0.4     0.6]
 [    8.6  4100.      1.   1450.     -1.4     2.6]
 [   13.6  6100.      0.   2450.     -2.4     3.6]
 [   18.6 11100.      2.   3450.     -1.4     1.6]
 [    6.6  3100.      1.    950.     -0.4     0.6]
 [   10.6  5100.      0.   1650.     -2.4     4.6]]


# Autovalores y autovectores usando covarianza

In [41]:
import numpy as np


# Covarianza
S = np.cov(Xc, rowvar=False)

# Autovalores y autovectores
eigvals, eigvecs = np.linalg.eigh(S)

# Ordenar de mayor a menor
idx = np.argsort(eigvals)[::-1]
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

print("Autovalores:", eigvals)
pc1 = eigvecs[:,0]
print("PC1 (Python):", pc1)



Autovalores: [50729013.2534    78386.389         4.4753        0.9392        0.1243
        0.0631]
PC1 (Python): [-0.0018 -0.9509 -0.0001 -0.3095  0.0003 -0.0004]


# Ortogonolizar


In [23]:
print("Ortogonalizar")

# Asegurar w normalizado (equivalente a w_norm en R)
w_norm = w / np.linalg.norm(w)

# Proyección de pc1 sobre w y resta: orto = pc1 - (pc1·w_norm) w_norm
orto = pc1 - np.dot(pc1, w_norm) * w_norm
print("Vector 'orto' (pc1 ortogonal a w):\n", np.round(orto, 6))

Ortogonalizar
Vector 'orto' (pc1 ortogonal a w):
 [-0.0017 -0.9509 -0.0001 -0.3095  0.0004 -0.0004]


# Proyectar


In [ ]:
# --------------------------------------------
# 2) Normalizar la dirección ortogonal (D2_antes)
# --------------------------------------------
D2_antes = orto / np.linalg.norm(orto)
print("\nD2_antes (dirección unitaria ortogonal a w):\n", np.round(D2_antes, 6))

# --------------------------------------------
# 3) Proyección de los datos sobre D2 (coordenadas)
# --------------------------------------------
# Equivalente a: D2 <- X %*% D2_antes en R
# "¿Qué hace?" -> multiplica la matriz de datos X (Nxd) por el vector D2_antes (dx1)
# y obtiene un vector (N,) con la coordenada de cada registro en el eje D2.
D2 = X @ D2_antes
print("\nD2 (coordenadas proyectadas, tamaño N):\n", np.round(D2, 4))

# Dimensiones


In [ ]:

# Armar DataFrame de dimensiones

dimensiones = pd.DataFrame({
    "id": datos["id"].values,
    "clase": clase,
    "D1": np.round(D1, 4),
    "D2": np.round(D2, 4)
})
print("\nDimensiones (primeras filas):")
print(dimensiones)

# Graficos de dimensiones


In [ ]:
import matplotlib.pyplot as plt
plt.figure()
for etiqueta in np.unique(clase):
    mask = (dimensiones["clase"] == etiqueta)
    plt.scatter(dimensiones.loc[mask, "D1"], dimensiones.loc[mask, "D2"], label=str(etiqueta))
    # Etiquetas de puntos con el id
    for _, row in dimensiones.loc[mask].iterrows():
        plt.text(row["D1"], row["D2"], str(int(row["id"])), fontsize=8, va='bottom')

plt.xlabel("Eje 1: LDA (w)")
plt.ylabel("Eje 2: Ortogonal a w (de PCA)")
plt.title("Proyección 2D: LDA (2 clases) + eje ortogonal")
plt.legend()
plt.tight_layout()
plt.show()

# Ejercicio ortogonal de dos vectores


In [ ]:
import numpy as np

# --- 1. Definir los vectores de 5 dimensiones ---
u = np.array([2, -1, 3, 0, 4])
v = np.array([1, 2, 1, 3, 1])

# --- 2. Implementar la fórmula de Gram-Schmidt ---

# 2a. Calcular el producto escalar (numerador: u · v)
u_dot_v = np.dot(u, v)
# Resultado: 7

# 2b. Calcular el cuadrado de la norma (denominador: ||v||^2)
v_norm_sq = np.dot(v, v)
# Resultado: 16

# 2c. Calcular el escalar (coeficiente de la proyección)
escalar = u_dot_v / v_norm_sq
# Resultado: 7/16 = 0.4375

# 2d. Calcular la Proyección Vectorial (Proy_v u)
proyeccion = escalar * v
# Resultado: [0.4375, 0.875, 0.4375, 1.3125, 0.4375]

# 2e. Calcular el vector Ortogonal (u_perpendicular = u - Proy_v u)
u_ortogonal = u - proyeccion

print("--- Vectores ---")
print(f"Vector V: {v}")
print(f"Vector U: {u}")
print(f"\nVector U ortogonal a V (Gram-Schmidt):")
print(u_ortogonal)

# --- 3. Comprobación de Ortogonalidad ---
# El producto escalar del resultado con V debe ser 0.
comprobacion = np.dot(u_ortogonal, v)

print("\n--- Comprobación ---")
# El resultado es un número extremadamente cercano a cero debido a la precisión.
print(f"Producto Escalar (U_ortogonal · V): {comprobacion}")

# Autovalores y primer componente de matriz centrada


In [ ]:
# Correlacion de v y y

In [ ]:
import numpy as np

# --- Vectores usados en el ejercicio Gram-Schmidt ---

# --- Verificación 1: Ortogonalidad (Producto Escalar) ---
# Debe ser CERO.
dot_product_check = np.dot(u_ortogonal, v)

print("--- Producto Escalar (u_ortogonal · v) ---")
print(f"Producto Escalar: {dot_product_check:.10f} (Cercano a 0, ¡es Ortogonal!)")
print("-" * 50)

# ----------------------------------------------------
# --- 2. CÁLCULO DE CORRELACIONES (np.corrcoef) ---
# ----------------------------------------------------

# Correlación entre u y v originales (Generalmente distinta de cero)
corr_uv = np.corrcoef(u, v)[0, 1]
print(f"1. Correlación entre u y v originales: {corr_uv:.4f}")

# Correlación entre u_ortogonal y v (Sale lejos de 0 por NO estar centrados)
corr_u_ortogonal_v = np.corrcoef(u_ortogonal, v)[0, 1]
print(f"2. Correlación de u_ortogonal y v:     {corr_u_ortogonal_v:.4f}  <-- NO CERO (Esto es lo que te sale)")

# --- 3. CORRECCIÓN: Centrar los vectores antes de correlacionar ---

# Centrar u_ortogonal
u_ortogonal_centrado = u_ortogonal - np.mean(u_ortogonal)

# Centrar v
v_centrado = v - np.mean(v)

# Correlación de los vectores centrados (¡Ahora sí debe dar CERO!)
corr_centrada = np.corrcoef(u_ortogonal_centrado, v_centrado)[0, 1]
print(f"3. Correlación con vectores CENTRADOS: {corr_centrada:.10f}  <-- ¡Correlación CERO!")

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(2025)
# Número de estudiantes
n = 10

# Dimensiones de salida (2D)
dimensiones_baja = 2

# Inicializar las coordenadas de baja dimensión (Y) aleatoriamente
y_inicial = np.random.randn(n, dimensiones_baja)


print(y_inicial)

# Calcular la matriz de distancias euclidianas al cuadrado en el espacio de baja dimensión
sum_y = np.sum(y_inicial**2, 1)
distancias_baja_dimension = np.add(np.add(-2 * np.dot(y_inicial, y_inicial.T), sum_y).T, sum_y)
# Matriz de similitudes de baja dimensión (Q)
# Sumar 1 a cada elemento de la matriz
Q_numerador = 1 / (1 + distancias_baja_dimension)
np.fill_diagonal(Q_numerador, 0.)  # La similitud de un punto consigo mismo es 0
# Calcular el denominador, la suma de todas las similitudes no diagonales
denominador_Q = np.sum(Q_numerador)
# Matriz Q final
Q_matriz_baja_dimension = pd.DataFrame(Q_numerador / denominador_Q)

print(Q_matriz_baja_dimension)
